## AutoGen Core

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

#### The fundamental principle

[Autogen Core](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/quickstart.html) decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a `Agent Runtime`.

There are 2 types:
- **[Standalone](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/core-concepts/architecture.html#standalone-agent-runtime)** - Standalone runtime is suitable for single-process applications where all agents are implemented in the same programming language and running in the same process
- **[Distributed](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/core-concepts/architecture.html#distributed-agent-runtime)** - Distributed runtime is suitable for multi-process applications where agents may be implemented in different programming languages and running on different machines.

This file focuses on standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.


In [ ]:
# Loading up required libraries
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv
from dataclasses import dataclass

In [ ]:
# loading up environment variables from env file
load_dotenv(override=True)

#### Defining Message object

This defines the structure we want for messages in our Agent framework.

In [ ]:
# Simple dataclass

@dataclass
class Message:
    content: str

#### Defining our Agent

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
- `agent.id.type` describes the kind of agent it is  
- `agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [ ]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("A Simple Agent")

    @message_handler
    async def on_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said {message.content}.")

#### Creating a Standalone runtime and register our agent type

In [ ]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

#### Starting runtime and sending a message

In [ ]:
runtime.start()

In [ ]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Hi there! How can I learn agentic ai?"), agent_id)
print(">>>", response.content)

#### Remember to always stop and close the runtime

In [ ]:
await runtime.stop()
await runtime.close()

#### Something more interesting!

Using an AgentChat Assistant!

In [ ]:
class LLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")
        self._delegate = AssistantAgent("LLMAgent", model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [ ]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await LLMAgent.register(runtime, "LLMAgent", lambda: LLMAgent())

In [ ]:
runtime.start()
response = await runtime.send_message(Message("How can I learn agentic ai?"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

#### Always remember to stop and close the runtime. This also applies if you want to re-run the cell above.

In [ ]:
await runtime.stop()
await runtime.close()

#### Introducing game player agents - having 3 agents interact!

In [ ]:
from pyexpat import model


class PlayerOneAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-1.5-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class PlayerTwoAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [ ]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
            instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
            message = Message(content=instruction)
            player_one = AgentId("player_one", "default")
            player_two = AgentId("player_two", "default")
            response1 = await self.send_message(message, player_one)
            response2 = await self.send_message(message, player_two)
            result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
            judgement = f"{JUDGE}{result}Who wins?"
            message = TextMessage(content=judgement, source="user")
            response = await self._delegate.on_messages([message], ctx.cancellation_token)
            return Message(content=result + response.chat_message.content)


In [ ]:
runtime = SingleThreadedAgentRuntime()
await PlayerOneAgent.register(runtime, "player_one", lambda: PlayerOneAgent("player_one"))
await PlayerTwoAgent.register(runtime, "player_two", lambda: PlayerTwoAgent("player_two"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [ ]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

In [ ]:
await runtime.stop()
await runtime.close()